# Characterizing CpG Methylation

To describe general metylation trends, irrespective of pCO<sub>2</sub> treatment in *C. virginica* gonad sequence data, I need to characterize individual CpG loci. Gavery and Roberts (2013) and Olson and Roberts (2013) define a CpG locus as methylated if at least half of the reads remained unconverted after bisulfite treatment. I will use information in a master `.cov` file to identify methylated CpG loci.

Another thing I will do is identify methylation islands by replicating [Jeong et al. 2018](https://academic.oup.com/gbe/article/10/10/2766/5098531). I will use [their script](https://github.com/soojinyilab/Methylation-Islands) but modify parameters to reflect differences in insect and *C. virginica* methylation.

1. Download coverage file
2. Limit to 5x coverage
3. Characterize methylation levels for loci
4. Characterize loci locations
5. Identify methylation islands

## 0. Prepare for analyses

## 0a. Set working directory

In [ ]:
pwd

In [ ]:
cd ../analyses/

In [3]:
!mkdir 2019-03-18-Characterizing-CpG-Methylation

In [ ]:
cd 2019-03-18-Characterizing-CpG-Methylation/

## 1. Obtain coverage files

In [4]:
#Download file from gannet. This file is a concatenation of coverage and methylation information for all samples
!wget http://gannet.fish.washington.edu/Atumefaciens/20190312_cvir_gonad_bismark/total_reads_bismark/cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov.gz

--2019-04-09 14:41:39--  http://gannet.fish.washington.edu/Atumefaciens/20190312_cvir_gonad_bismark/total_reads_bismark/cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov.gz
Resolving gannet.fish.washington.edu... 128.95.149.52
Connecting to gannet.fish.washington.edu|128.95.149.52|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94181669 (90M) [application/x-gzip]
Saving to: 'cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov.gz'

cvir_bsseq_all_pe_R 100%[===================>]  89.82M  75.1MB/s    in 1.2s    

2019-04-09 14:41:40 (75.1 MB/s) - 'cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov.gz' saved [94181669/94181669]



In [8]:
#Unzip the coverage file
!gunzip *cov.gz

In [4]:
#Confirm file was unzipped
!ls *cov

cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov


In [4]:
#See what the file looks like. 
#Columns: <chromosome> <start position> <end position> <methylation percentage> <count methylated> <count unmethylated>
!head cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov

NC_007175.2	49	49	1.25	2	158
NC_007175.2	50	50	0	0	15
NC_007175.2	51	51	1.18343195266272	2	167
NC_007175.2	52	52	0	0	18
NC_007175.2	88	88	1.02459016393443	5	483
NC_007175.2	89	89	1.38888888888889	5	355
NC_007175.2	100	100	0	0	1
NC_007175.2	129	129	0	0	1
NC_007175.2	147	147	1.99115044247788	18	886
NC_007175.2	148	148	2.29885057471264	6	255


In [6]:
#See how many loci have data
!awk '{if ($5+$6 >= 1) { print $1, $2-1, $3, $4, $5+$6}}' cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov \
| wc -l

 14026131


14,026,131 CpGs have data, which is close to the 14,458,703 CG motifs in the *C. virginica* genome.

## 2. Limit to 5x coverage

In [11]:
#If total coverage (count methylated + unmethylated) is greater than 5
#then print the chromosome, start pos -1, stop pos, percent methylation, and total coverage
#Save output as new file
!awk '{if ($5+$6 >= 5) { print $1, $2-1, $3, $4, $5+$6}}' cvir_bsseq_all_pe_R1_bismark_bt2_pe.bismark.cov \
> 2019-04-09-All-5x-CpGs.bedgraph

In [12]:
#Check columns for one of the file: <chromosome> <start position> <stop position> <percent methylation> <coverage>
!head 2019-04-09-All-5x-CpGs.bedgraph

NC_007175.2 48 49 1.25 160
NC_007175.2 49 50 0 15
NC_007175.2 50 51 1.18343195266272 169
NC_007175.2 51 52 0 18
NC_007175.2 87 88 1.02459016393443 488
NC_007175.2 88 89 1.38888888888889 360
NC_007175.2 146 147 1.99115044247788 904
NC_007175.2 147 148 2.29885057471264 261
NC_007175.2 173 174 0 5
NC_007175.2 192 193 1.25786163522013 795


In [58]:
#Count loci with 5x coverage
!wc -l 2019-04-09-All-5x-CpGs.bedgraph

 4304257 2019-04-09-All-5x-CpGs.bedgraph


I have data for 4,304,257 CpG loci with 5x coverge.

In [59]:
#Replace delimiters to save .bedgraph as .csv
!awk '{print $1","$2","$3","$4 }' 2019-04-09-All-5x-CpGs.bedgraph \
> 2019-04-09-All-5x-CpGs.csv

In [60]:
#Confirm .csv creation
!head 2019-04-09-All-5x-CpGs.csv

NC_007175.2,48,49,1.25
NC_007175.2,49,50,0
NC_007175.2,50,51,1.18343195266272
NC_007175.2,51,52,0
NC_007175.2,87,88,1.02459016393443
NC_007175.2,88,89,1.38888888888889
NC_007175.2,146,147,1.99115044247788
NC_007175.2,147,148,2.29885057471264
NC_007175.2,173,174,0
NC_007175.2,192,193,1.25786163522013


## 3. Characterize methylation levels for loci

Olson and Roberts (2014) define the following categories for CpG methylation:

- Methylated (50% methylation and above)
- Sparsely methylated (0-50% methylated)
- Unmethylated (0% methylation)

I will slightly modify this since I have multiple samples:

- Methylated (50% methylation and above)
- Sparsely methylated (10-50% methylated)
- Unmethylated (10% methylation and below)

### 3a. Methylated loci

In [19]:
#If percent methylation is greater or equal to 50, then save the loci information
!awk '{if ($4 >= 50) { print $1, $2, $3, $4 }}' 2019-04-09-All-5x-CpGs.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph

In [48]:
#Confirm methylated loci were saved
!head 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph

NC_035780.1 9253 9254 60
NC_035780.1 9637 9638 60
NC_035780.1 9657 9658 50
NC_035780.1 10089 10090 71.4285714285714
NC_035780.1 10331 10332 80
NC_035780.1 11692 11693 80
NC_035780.1 11706 11707 80
NC_035780.1 11711 11712 80
NC_035780.1 12686 12687 69.2307692307692
NC_035780.1 12758 12759 80


In [21]:
#Count methylated loci
!wc -l 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph

 3181904 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph


In [53]:
#Replace delimiters to save .bedgraph as .csv
!awk '{print $1","$2","$3","$4 }' 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Methylated.csv

In [54]:
#Check .csv was saved
!head 2019-04-09-All-5x-CpG-Loci-Methylated.csv

NC_035780.1,9253,9254,60
NC_035780.1,9637,9638,60
NC_035780.1,9657,9658,50
NC_035780.1,10089,10090,71.4285714285714
NC_035780.1,10331,10332,80
NC_035780.1,11692,11693,80
NC_035780.1,11706,11707,80
NC_035780.1,11711,11712,80
NC_035780.1,12686,12687,69.2307692307692
NC_035780.1,12758,12759,80


### 3b. Sparsely methylated loci

In [33]:
%%bash
awk '{if ($4 < 50) { print $1, $2, $3, $4}}' 2019-04-09-All-5x-CpGs.bedgraph \
| awk '{if ($4 > 10) { print $1, $2, $3, $4 }}' \
> 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph

In [34]:
#Confirm sparsely methylated loci were saved
!head 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph

NC_007175.2 1506 1507 16.6666666666667
NC_007175.2 1820 1821 20
NC_007175.2 2128 2129 11.7647058823529
NC_007175.2 4841 4842 15
NC_007175.2 13069 13070 20
NC_035780.1 421 422 14.2857142857143
NC_035780.1 1101 1102 12.5
NC_035780.1 1540 1541 16.6666666666667
NC_035780.1 3468 3469 16.6666666666667
NC_035780.1 9254 9255 28.5714285714286


In [35]:
#Count sparsely methylated loci
!wc -l 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph

  481788 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph


### 3c. Unmethylated loci

In [36]:
!awk '{if ($4 <= 10) { print $1, $2, $3, $4 }}' 2019-04-09-All-5x-CpGs.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph

In [37]:
#Confirm unmethylated loci were saved
!head 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph

NC_007175.2 48 49 1.25
NC_007175.2 49 50 0
NC_007175.2 50 51 1.18343195266272
NC_007175.2 51 52 0
NC_007175.2 87 88 1.02459016393443
NC_007175.2 88 89 1.38888888888889
NC_007175.2 146 147 1.99115044247788
NC_007175.2 147 148 2.29885057471264
NC_007175.2 173 174 0
NC_007175.2 192 193 1.25786163522013


In [38]:
#Count unmethylated loci
!wc -l 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph

  640565 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph


## 4. Characterize loci locations

My final step is to characterize the location of various loci categories in the genome. I will use `intersectBed` to find overlaps between all 5x CpGs, methylated loci, sparsely methylated loci, and unmethylated loci with exons, introns, mRNA coding regions, transposable elements, and putative promoter regions.

### 4a. Create `.bed` files

#### All 5x CpGs

In [69]:
%%bash
awk '{print $1"\t"$2"\t"$3}' 2019-04-09-All-5x-CpGs.bedgraph \
> 2019-04-09-All-5x-CpGs.bed

In [70]:
#Confirm file creation
!head 2019-04-09-All-5x-CpGs.bed

NC_007175.2	48	49
NC_007175.2	49	50
NC_007175.2	50	51
NC_007175.2	51	52
NC_007175.2	87	88
NC_007175.2	88	89
NC_007175.2	146	147
NC_007175.2	147	148
NC_007175.2	173	174
NC_007175.2	192	193


#### Methylated loci

In [8]:
%%bash
awk '{print $1"\t"$2"\t"$3}' 2019-04-09-All-5x-CpG-Loci-Methylated.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Methylated.bed

In [4]:
#Confirm file creation
!head 2019-04-09-All-5x-CpG-Loci-Methylated.bed

NC_035780.1	9253	9254
NC_035780.1	9637	9638
NC_035780.1	9657	9658
NC_035780.1	10089	10090
NC_035780.1	10331	10332
NC_035780.1	11692	11693
NC_035780.1	11706	11707
NC_035780.1	11711	11712
NC_035780.1	12686	12687
NC_035780.1	12758	12759


#### Sparsely methylated loci

In [5]:
%%bash
awk '{print $1"\t"$2"\t"$3}' 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bed

In [6]:
#Confirm file creation
!head 2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bed

NC_007175.2	1506	1507
NC_007175.2	1820	1821
NC_007175.2	2128	2129
NC_007175.2	4841	4842
NC_007175.2	13069	13070
NC_035780.1	421	422
NC_035780.1	1101	1102
NC_035780.1	1540	1541
NC_035780.1	3468	3469
NC_035780.1	9254	9255


#### Unmethylated loci

In [7]:
%%bash
awk '{print $1"\t"$2"\t"$3}' 2019-04-09-All-5x-CpG-Loci-Unmethylated.bedgraph \
> 2019-04-09-All-5x-CpG-Loci-Unmethylated.bed

In [8]:
#Confirm file creation
!head 2019-04-09-All-5x-CpG-Loci-Unmethylated.bed

NC_007175.2	48	49
NC_007175.2	49	50
NC_007175.2	50	51
NC_007175.2	51	52
NC_007175.2	87	88
NC_007175.2	88	89
NC_007175.2	146	147
NC_007175.2	147	148
NC_007175.2	173	174
NC_007175.2	192	193


### 4b. Set variable paths

In [7]:
bedtoolsDirectory = "/Users/Shared/bioinformatics/bedtools2/bin/"

In [8]:
all5xCpGs = "2019-04-09-All-5x-CpGs.bed"

In [9]:
methylatedLoci = "2019-04-09-All-5x-CpG-Loci-Methylated.bed"

In [10]:
sparselyMethylatedLoci = "2019-04-09-All-5x-CpG-Loci-Sparsely-Methylated.bed"

In [11]:
unmethylatedLoci = "2019-04-09-All-5x-CpG-Loci-Unmethylated.bed"

In [17]:
putativePromoters = "../../genome-feature-tracks/2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Promoter-Track.bed"

In [ ]:
exonUTR = "../../genome-feature-tracks/C_virginica-3.0_Gnomon_exonUTR_yrv.gff3"

In [12]:
exonList = "../../genome-feature-tracks/C_virginica-3.0_Gnomon_exon_sorted_yrv.bed"

In [13]:
intronList = "../../genome-feature-tracks/C_virginica-3.0_Gnomon_intron_yrv.bed"

In [14]:
geneList = "../../genome-feature-tracks/C_virginica-3.0_Gnomon_gene_sorted_yrv.bed"

In [15]:
transposableElementsAll = "../../genome-feature-tracks/C_virginica-3.0_TE-all.gff"

In [ ]:
intergenic = "../../genome-feature-tracks/C_virginica-3.0_Gnomon_intergenic_yrv.gff3"

### 4c. Exons

#### All 5x CpGs

In [16]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {exonList} \
| wc -l
!echo "all 5x CpG loci overlaps with exons"

 1366779
all 5x CpG loci overlaps with exons


In [17]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {exonList} \
> 2019-05-29-All5xCpGs-Exon.txt

In [18]:
!head 2019-05-29-All5xCpGs-Exon.txt

NC_035780.1	28992	28993	NC_035780.1	28961	29073
NC_035780.1	29001	29002	NC_035780.1	28961	29073
NC_035780.1	30723	30724	NC_035780.1	30524	31557
NC_035780.1	30765	30766	NC_035780.1	30524	31557
NC_035780.1	30811	30812	NC_035780.1	30524	31557
NC_035780.1	30906	30907	NC_035780.1	30524	31557
NC_035780.1	30932	30933	NC_035780.1	30524	31557
NC_035780.1	30935	30936	NC_035780.1	30524	31557
NC_035780.1	31017	31018	NC_035780.1	30524	31557
NC_035780.1	31018	31019	NC_035780.1	30524	31557


#### Methylated loci

In [19]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {exonList} \
| wc -l
!echo "methylated loci overlaps with exons"

 1013691
methylated loci overlaps with exons


In [20]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {exonList} \
> 2019-05-29-MethLoci-Exon.txt

In [21]:
!head 2019-05-29-MethLoci-Exon.txt

NC_035780.1	100558	100559	NC_035780.1	100554	100661
NC_035780.1	100559	100560	NC_035780.1	100554	100661
NC_035780.1	100575	100576	NC_035780.1	100554	100661
NC_035780.1	100576	100577	NC_035780.1	100554	100661
NC_035780.1	100581	100582	NC_035780.1	100554	100661
NC_035780.1	100582	100583	NC_035780.1	100554	100661
NC_035780.1	100634	100635	NC_035780.1	100554	100661
NC_035780.1	100635	100636	NC_035780.1	100554	100661
NC_035780.1	100643	100644	NC_035780.1	100554	100661
NC_035780.1	100644	100645	NC_035780.1	100554	100661


#### Sparsely methylated loci

In [22]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {exonList} \
| wc -l
!echo "sparsely methylated loci overlaps with exons"

  105871
sparsely methylated loci overlaps with exons


In [23]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {exonList} \
> 2019-05-29-SparseMethLoci-Exon.txt

In [24]:
!head 2019-05-29-SparseMethLoci-Exon.txt

NC_035780.1	31078	31079	NC_035780.1	30524	31557
NC_035780.1	85755	85756	NC_035780.1	85606	85777
NC_035780.1	94754	94755	NC_035780.1	94571	95254
NC_035780.1	106236	106237	NC_035780.1	106004	106460
NC_035780.1	204528	204529	NC_035780.1	204243	204795
NC_035780.1	207401	207402	NC_035780.1	207388	207743
NC_035780.1	207423	207424	NC_035780.1	207388	207743
NC_035780.1	207472	207473	NC_035780.1	207388	207743
NC_035780.1	223409	223410	NC_035780.1	223311	223637
NC_035780.1	223416	223417	NC_035780.1	223311	223637


#### Unmethylated loci

In [25]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {exonList} \
| wc -l
!echo "unmethylated loci overlaps with exons"

  247217
unmethylated loci overlaps with exons


In [26]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {exonList} \
> 2019-05-29-UnMethLoci-Exon.txt

In [27]:
!head 2019-05-29-UnMethLoci-Exon.txt

NC_035780.1	28992	28993	NC_035780.1	28961	29073
NC_035780.1	29001	29002	NC_035780.1	28961	29073
NC_035780.1	30723	30724	NC_035780.1	30524	31557
NC_035780.1	30765	30766	NC_035780.1	30524	31557
NC_035780.1	30811	30812	NC_035780.1	30524	31557
NC_035780.1	30906	30907	NC_035780.1	30524	31557
NC_035780.1	30932	30933	NC_035780.1	30524	31557
NC_035780.1	30935	30936	NC_035780.1	30524	31557
NC_035780.1	31017	31018	NC_035780.1	30524	31557
NC_035780.1	31018	31019	NC_035780.1	30524	31557


### 4d. Introns

#### All 5x CpG

In [28]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {intronList} \
| wc -l
!echo "all 5x CpG loci overlaps with introns"

 1884429
all 5x CpG loci overlaps with introns


In [29]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {intronList} \
> 2019-05-29-All5xCpGs-Intron.txt

In [30]:
!head 2019-05-29-All5xCpGs-Intron.txt

NC_035780.1	29412	29413	NC_035780.1	29073	30523
NC_035780.1	31940	31941	NC_035780.1	31887	31976
NC_035780.1	44372	44373	NC_035780.1	44358	45912
NC_035780.1	45142	45143	NC_035780.1	44358	45912
NC_035780.1	45542	45543	NC_035780.1	44358	45912
NC_035780.1	46515	46516	NC_035780.1	46506	64122
NC_035780.1	47583	47584	NC_035780.1	46506	64122
NC_035780.1	47590	47591	NC_035780.1	46506	64122
NC_035780.1	47651	47652	NC_035780.1	46506	64122
NC_035780.1	47679	47680	NC_035780.1	46506	64122


#### Methylated loci

In [31]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {intronList} \
| wc -l
!echo "methylated loci overlaps with introns"

 1504791
methylated loci overlaps with introns


In [32]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {intronList} \
> 2019-05-29-MethLoci-Intron.txt

In [33]:
!head 2019-05-29-MethLoci-Intron.txt

NC_035780.1	29412	29413	NC_035780.1	29073	30523
NC_035780.1	87531	87532	NC_035780.1	85777	88422
NC_035780.1	87541	87542	NC_035780.1	85777	88422
NC_035780.1	87590	87591	NC_035780.1	85777	88422
NC_035780.1	87595	87596	NC_035780.1	85777	88422
NC_035780.1	100664	100665	NC_035780.1	100661	104928
NC_035780.1	100665	100666	NC_035780.1	100661	104928
NC_035780.1	100917	100918	NC_035780.1	100661	104928
NC_035780.1	100975	100976	NC_035780.1	100661	104928
NC_035780.1	101305	101306	NC_035780.1	100661	104928


#### Sparsely methylated loci

In [34]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {intronList} \
| wc -l
!echo "sparsely methylated loci overlaps with introns"

  211143
sparsely methylated loci overlaps with introns


In [35]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {intronList} \
> 2019-05-29-SparseMethLoci-Intron.txt

In [36]:
!head 2019-05-29-SparseMethLoci-Intron.txt

NC_035780.1	45142	45143	NC_035780.1	44358	45912
NC_035780.1	45542	45543	NC_035780.1	44358	45912
NC_035780.1	48914	48915	NC_035780.1	46506	64122
NC_035780.1	48928	48929	NC_035780.1	46506	64122
NC_035780.1	48940	48941	NC_035780.1	46506	64122
NC_035780.1	87599	87600	NC_035780.1	85777	88422
NC_035780.1	87607	87608	NC_035780.1	85777	88422
NC_035780.1	103272	103273	NC_035780.1	100661	104928
NC_035780.1	104332	104333	NC_035780.1	100661	104928
NC_035780.1	105767	105768	NC_035780.1	105614	106003


#### Unmethylated loci

In [37]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {intronList} \
| wc -l
!echo "unmethylated loci overlaps with introns"

  168495
unmethylated loci overlaps with introns


In [38]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {intronList} \
> 2019-05-29-UnMethLoci-Intron.txt

In [39]:
!head 2019-05-29-UnMethLoci-Intron.txt

NC_035780.1	31940	31941	NC_035780.1	31887	31976
NC_035780.1	44372	44373	NC_035780.1	44358	45912
NC_035780.1	46515	46516	NC_035780.1	46506	64122
NC_035780.1	47583	47584	NC_035780.1	46506	64122
NC_035780.1	47590	47591	NC_035780.1	46506	64122
NC_035780.1	47651	47652	NC_035780.1	46506	64122
NC_035780.1	47679	47680	NC_035780.1	46506	64122
NC_035780.1	48094	48095	NC_035780.1	46506	64122
NC_035780.1	48108	48109	NC_035780.1	46506	64122
NC_035780.1	48114	48115	NC_035780.1	46506	64122


### 4e. Exon UTR

#### All 5x CpGs

In [ ]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {exonUTR} \
| wc -l
!echo "all 5x CpG loci overlaps with exon UTR"

In [ ]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {exonUTR} \
> 2019-05-29-All5xCpGs-ExonUTR.txt

In [ ]:
!head 2019-05-29-All5xCpGs-ExonUTR.txt

#### Methylated loci

In [ ]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {exonUTR} \
| wc -l
!echo "methylated loci overlaps with exon UTR"

In [ ]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {exonUTR} \
> 2019-05-29-MethLoci-ExonUTR.txt

In [ ]:
!head 2019-05-29-MethLoci-ExonUTR.txt

#### Sparsely methylated loci

In [ ]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {exonUTR} \
| wc -l
!echo "sparsely methylated loci overlaps with exon UTR"

In [ ]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {exonUTR} \
> 2019-05-29-SparseMethLoci-ExonUTR.txt

In [ ]:
!head 2019-05-29-SparseMethLoci-ExonUTR.txt

#### Unmethylated loci

In [ ]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {exonUTR} \
| wc -l
!echo "unmethylated loci overlaps with exon UTR"

In [ ]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {exonUTR} \
> 2019-05-29-UnMethLoci-ExonUTR.txt

In [ ]:
!head 2019-05-29-UnMethLoci-ExonUTR.txt

### 4f. Genes

#### All 5x CpGs

In [40]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {geneList} \
| wc -l
!echo "all 5x CpG loci overlaps with genes"

 3255049
all 5x CpG loci overlaps with genes


In [41]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {geneList} \
> 2019-05-29-All5xCpGs-Genes.txt

In [42]:
!head 2019-05-29-All5xCpGs-Genes.txt

NC_035780.1	28992	28993	NC_035780.1	28961	33324
NC_035780.1	29001	29002	NC_035780.1	28961	33324
NC_035780.1	29412	29413	NC_035780.1	28961	33324
NC_035780.1	30723	30724	NC_035780.1	28961	33324
NC_035780.1	30765	30766	NC_035780.1	28961	33324
NC_035780.1	30811	30812	NC_035780.1	28961	33324
NC_035780.1	30906	30907	NC_035780.1	28961	33324
NC_035780.1	30932	30933	NC_035780.1	28961	33324
NC_035780.1	30935	30936	NC_035780.1	28961	33324
NC_035780.1	31017	31018	NC_035780.1	28961	33324


In [44]:
!cut -f6 2019-05-29-All5xCpGs-Genes.txt| sort | uniq -c | wc -l
!echo "unique genes represented in overlaps"

   33126
unique genes represented in overlaps


#### Methylated loci

In [45]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {geneList} \
| wc -l
!echo "methylated loci overlaps with genes"

 2521653
methylated loci overlaps with genes


In [47]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {geneList} \
> 2019-05-29-MethLoci-Genes.txt

In [48]:
!head 2019-05-29-MethLoci-Genes.txt

NC_035780.1	29412	29413	NC_035780.1	28961	33324
NC_035780.1	87531	87532	NC_035780.1	85606	95254
NC_035780.1	87541	87542	NC_035780.1	85606	95254
NC_035780.1	87590	87591	NC_035780.1	85606	95254
NC_035780.1	87595	87596	NC_035780.1	85606	95254
NC_035780.1	100558	100559	NC_035780.1	99840	106460
NC_035780.1	100559	100560	NC_035780.1	99840	106460
NC_035780.1	100575	100576	NC_035780.1	99840	106460
NC_035780.1	100576	100577	NC_035780.1	99840	106460
NC_035780.1	100581	100582	NC_035780.1	99840	106460


In [52]:
!cut -f6 2019-05-29-MethLoci-Genes.txt| sort | uniq -c | wc -l
!echo "unique genes represented in overlaps"

   25496
unique genes represented in overlaps


#### Sparsely methylated loci

In [53]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {geneList} \
| wc -l
!echo "sparsely methylated loci overlaps with genes"

  317249
sparsely methylated loci overlaps with genes


In [54]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {geneList} \
> 2019-05-29-SparseMethLoci-Genes.txt

In [55]:
!head 2019-05-29-SparseMethLoci-Genes.txt

NC_035780.1	31078	31079	NC_035780.1	28961	33324
NC_035780.1	45142	45143	NC_035780.1	43111	66897
NC_035780.1	45542	45543	NC_035780.1	43111	66897
NC_035780.1	48914	48915	NC_035780.1	43111	66897
NC_035780.1	48928	48929	NC_035780.1	43111	66897
NC_035780.1	48940	48941	NC_035780.1	43111	66897
NC_035780.1	85755	85756	NC_035780.1	85606	95254
NC_035780.1	87599	87600	NC_035780.1	85606	95254
NC_035780.1	87607	87608	NC_035780.1	85606	95254
NC_035780.1	94754	94755	NC_035780.1	85606	95254


In [57]:
!cut -f6 2019-05-29-SparseMethLoci-Genes.txt| sort | uniq -c | wc -l
!echo "unique genes represesnted in overlaps"

   26953
unique genes represesnted in overlaps


#### Unmethylated loci

In [58]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {geneList} \
| wc -l
!echo "unmethylated loci overlaps with genes"

  416147
unmethylated loci overlaps with genes


In [59]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {geneList} \
> 2019-05-29-UnMethLoci-Genes.txt

In [60]:
!head 2019-05-29-UnMethLoci-Genes.txt

NC_035780.1	28992	28993	NC_035780.1	28961	33324
NC_035780.1	29001	29002	NC_035780.1	28961	33324
NC_035780.1	30723	30724	NC_035780.1	28961	33324
NC_035780.1	30765	30766	NC_035780.1	28961	33324
NC_035780.1	30811	30812	NC_035780.1	28961	33324
NC_035780.1	30906	30907	NC_035780.1	28961	33324
NC_035780.1	30932	30933	NC_035780.1	28961	33324
NC_035780.1	30935	30936	NC_035780.1	28961	33324
NC_035780.1	31017	31018	NC_035780.1	28961	33324
NC_035780.1	31018	31019	NC_035780.1	28961	33324


In [61]:
!cut -f6 2019-05-29-UnMethLoci-Genes.txt| sort | uniq -c | wc -l
!echo "unique genes represented in overlaps"

   27753
unique genes represented in overlaps


### 4g. Putative promoters

#### All 5x CpGs

In [18]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {putativePromoters} \
| wc -l
!echo "all 5x CpG loci overlaps with putative promoters"

  176156
all 5x CpG loci overlaps with putative promoters


In [19]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {putativePromoters} \
> 2019-05-29-All5xCpGs-Putative-Promoters.txt

In [20]:
!head 2019-05-29-All5xCpGs-Putative-Promoters.txt

NC_035780.1	27969	27970	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	27979	27980	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	28082	28083	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:1

#### Methylated loci

In [21]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {putativePromoters} \
| wc -l
!echo "methylated loci overlaps with putative promoters"

  106111
methylated loci overlaps with putative promoters


In [22]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {putativePromoters} \
> 2019-05-29-MethLoci-Putative-Promoters.txt

In [23]:
!head 2019-05-29-MethLoci-Putative-Promoters.txt

NC_035780.1	27969	27970	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	27979	27980	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	28082	28083	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:1

#### Sparsely methylated loci

In [24]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {putativePromoters} \
| wc -l
!echo "sparsely methylated loci overlaps with putative promoters"

   22870
sparsely methylated loci overlaps with putative promoters


In [25]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {putativePromoters} \
> 2019-05-29-SparseMethLoci-Putative-Promoters.txt

In [26]:
!head 2019-05-29-SparseMethLoci-Putative-Promoters.txt

NC_035780.1	95674	95675	NC_035780.1	Gnomon	mRNA	95255	96254	.	-	.	ID=rna4;Parent=gene3;Dbxref=GeneID:111112434,Genbank:XM_022449924.1;Name=XM_022449924.1;gbkey=mRNA;gene=LOC111112434;model_evidence=Supporting evidence includes similarity to: 7 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 13 samples with support for all annotated introns;product=homeobox protein Hox-B7-like;transcript_id=XM_022449924.1
NC_035780.1	99251	99252	NC_035780.1	Gnomon	mRNA	98840	99839	.	+	.	ID=rna5;Parent=gene4;Dbxref=GeneID:111120752,Genbank:XM_022461698.1;Name=XM_022461698.1;gbkey=mRNA;gene=LOC111120752;model_evidence=Supporting evidence includes similarity to: 10 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 27 samples with support for all annotated introns;product=ribulose-phosphate 3-epimerase-like;transcript_id=XM_022461698.1
NC_035780.1	232223	232224	NC_035780.1	Gnomon	mRNA	231965	232964	.	-	.	ID=r

#### Unmethylated loci

In [27]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {putativePromoters} \
| wc -l
!echo "unmethylated loci overlaps with putative promoters"

   47175
unmethylated loci overlaps with putative promoters


In [28]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {putativePromoters} \
> 2019-05-29-UnMethLoci-Putative-Promoters.txt

In [29]:
!head 2019-05-29-UnMethLoci-Putative-Promoters.txt

NC_035780.1	28859	28860	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	28924	28925	NC_035780.1	Gnomon	mRNA	27961	28960	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	46515	46516	NC_035780.1	Gnomon	mRNA	46507	47506	.	-	.	ID=rna3;Parent=gene2;Dbxref=GeneID:1

### 4h. Transposable elements (all)

#### All 5x CpGs

In [62]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {transposableElementsAll} \
| wc -l
!echo "all 5x CpG loci overlaps with transposable elements (all)"

 1011883
all 5x CpG loci overlaps with transposable elements (all)


In [63]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {transposableElementsAll} \
> 2019-05-29-All5xCpGs-TE-All.txt

In [64]:
!head 2019-05-29-All5xCpGs-TE-All.txt

NC_007175.2	263	264	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	264	265	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	265	266	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	266	267	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	295	296	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	331	332	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	332	333	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	366	367	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	367	368	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	397	398

#### Methylated loci

In [65]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {transposableElementsAll} \
| wc -l
!echo "methylated loci overlaps with transposable elements (all)"

  755222
methylated loci overlaps with transposable elements (all)


In [66]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {transposableElementsAll} \
> 2019-05-29-MethLoci-TE-All.txt

In [67]:
!head 2019-05-29-MethLoci-TE-All.txt

NC_035780.1	9253	9254	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	19631	19632	NC_035780.1	RepeatMasker	similarity	19431	19866	23.3	-	.	Target "Motif:Crypton-N19_CGi" 580 1033
NC_035780.1	19741	19742	NC_035780.1	RepeatMasker	similarity	19431	19866	23.3	-	.	Target "Motif:Crypton-N19_CGi" 580 1033
NC_035780.1	37557	37558	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37581	37582	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37604	37605	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37611	37612	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37618	37619	NC_035780.1	RepeatMasker	similarity	37557	37890	12.9	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	37622	37623	NC_035780.1	RepeatMasker	

#### Sparsely methylated loci

In [68]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {transposableElementsAll} \
| wc -l
!echo "sparsely methylated loci overlaps with transposable elements (all)"

  155293
sparsely methylated loci overlaps with transposable elements (all)


In [69]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {transposableElementsAll} \
> 2019-05-29-SparseMethLoci-TE-All.txt

In [70]:
!head 2019-05-29-SparseMethLoci-TE-All.txt

NC_007175.2	1820	1821	NC_007175.2	RepeatMasker	similarity	1728	1947	26.1	-	.	Target "Motif:REP-6_LMi" 14320 14534
NC_007175.2	2128	2129	NC_007175.2	RepeatMasker	similarity	2129	2367	20.5	-	.	Target "Motif:REP-6_LMi" 13886 14118
NC_035780.1	9254	9255	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9266	9267	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9267	9268	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9297	9298	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9298	9299	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9301	9302	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780.1	9302	9303	NC_035780.1	RepeatMasker	similarity	9223	9562	26.9	-	.	Target "Motif:DNA-19_CGi" 1 332
NC_035780

#### Unmethylated loci

In [71]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {transposableElementsAll} \
| wc -l
!echo "unmethylated loci overlaps with transposable elements (all)"

  101368
unmethylated loci overlaps with transposable elements (all)


In [72]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {transposableElementsAll} \
> 2019-05-29-UnMethLoci-TE-All.txt

In [73]:
!head 2019-05-29-UnMethLoci-TE-All.txt

NC_007175.2	263	264	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	264	265	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	265	266	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	266	267	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	295	296	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	331	332	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	332	333	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	366	367	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	367	368	NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	397	398

### 4i. Intergenic regions

#### All 5x CpGs

In [ ]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {all5xCpGs} \
-b {intergenic} \
| wc -l
!echo "all 5x CpG loci overlaps with intergenic regions"

In [ ]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {all5xCpGs} \
-b {intergenic} \
> 2019-05-29-All5xCpGs-intergenic.txt

In [ ]:
!head 2019-05-29-All5xCpGs-intergenic.txt

#### Methylated loci

In [ ]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylatedLoci} \
-b {intergenic} \
| wc -l
!echo "methylated loci overlaps with intergenic regions"

In [ ]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {methylatedLoci} \
-b {intergenic} \
> 2019-05-29-MethLoci-intergenic.txt

In [ ]:
!head 2019-05-29-MethLoci-intergenic.txt

#### Sparsely methylated loci

In [ ]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {sparselyMethylatedLoci} \
-b {intergenic} \
| wc -l
!echo "sparsely methylated loci overlaps with intergenic regions"

In [ ]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {sparselyMethylatedLoci} \
-b {intergenic} \
> 2019-05-29-SparseMethLoci-intergenic.txt

In [ ]:
!head 2019-05-29-SparseMethLoci-intergenic.txt

#### Unmethylated loci

In [ ]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {unmethylatedLoci} \
-b {intergenic} \
| wc -l
!echo "unmethylated loci overlaps with intergenic regions"

In [ ]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {unmethylatedLoci} \
-b {intergenic} \
> 2019-05-29-UnMethLoci-intergenic.txt

In [ ]:
!head 2019-05-29-UnMethLoci-intergenic.txt

### 4j. No overlaps

#### All 5x CpGs

In [30]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {all5xCpGs} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
| wc -l
!echo "all 5x CpG loci do not overlap with exons, introns, transposable elements (all), or putative promoters"

  603597
all 5x CpG loci do not overlap with exons, introns, transposable elements (all), or putative promoters


In [31]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {all5xCpGs} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
> 2019-05-29-All5xCpGs-NoOverlaps.txt

In [32]:
!head 2019-05-29-All5xCpGs-NoOverlaps.txt

NC_007175.2	48	49
NC_007175.2	49	50
NC_007175.2	50	51
NC_007175.2	51	52
NC_007175.2	87	88
NC_007175.2	88	89
NC_007175.2	146	147
NC_007175.2	147	148
NC_007175.2	173	174
NC_007175.2	192	193


#### Methylated loci

In [33]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {methylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
| wc -l
!echo "methylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters"

  372047
methylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters


In [34]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {methylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
> 2019-05-29-MethLoci-NoOverlaps.txt

In [35]:
!head 2019-05-29-MethLoci-NoOverlaps.txt

NC_035780.1	9637	9638
NC_035780.1	9657	9658
NC_035780.1	10089	10090
NC_035780.1	10331	10332
NC_035780.1	11692	11693
NC_035780.1	11706	11707
NC_035780.1	11711	11712
NC_035780.1	12686	12687
NC_035780.1	12758	12759
NC_035780.1	13486	13487


#### Sparsely methylated loci

In [36]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {sparselyMethylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
| wc -l
!echo "sparsely methylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters"

   84582
sparsely methylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters


In [37]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {sparselyMethylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
> 2019-05-29-SparseMethLoci-NoOverlaps.txt

In [38]:
!head 2019-05-29-SparseMethLoci-NoOverlaps.txt

NC_007175.2	1506	1507
NC_007175.2	4841	4842
NC_007175.2	13069	13070
NC_035780.1	421	422
NC_035780.1	1101	1102
NC_035780.1	1540	1541
NC_035780.1	3468	3469
NC_035780.1	9789	9790
NC_035780.1	9832	9833
NC_035780.1	9854	9855


#### Unmethylated loci

In [39]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {unmethylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
| wc -l
!echo "unmethylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters"

  146968
unmethylated loci do not overlap with exons, introns, transposable elements (all), or putative promoters


In [40]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {unmethylatedLoci} \
-b {exonList} {intronList} {transposableElementsAll} {putativePromoters} \
> 2019-05-29-UnMethLoci-NoOverlaps.txt

In [41]:
!head 2019-05-29-UnMethLoci-NoOverlaps.txt

NC_007175.2	48	49
NC_007175.2	49	50
NC_007175.2	50	51
NC_007175.2	51	52
NC_007175.2	87	88
NC_007175.2	88	89
NC_007175.2	146	147
NC_007175.2	147	148
NC_007175.2	173	174
NC_007175.2	192	193


## 5. Identify methylation islands

To identify methylation islands using the method from Jeong et al. (2018), I need to define:

- starting size of the methylation window: 500 bp
- minimum fraction of methylated CpGs required within the window to be accepted: 0.02
- step size to extend the accepted window as long as the mCpG fraction is met: 50 bp
- mCpG file: input with mCpG chromosome and bp position

### 5a. Create mCpG file

In [ ]:
#Modify mCpG file by removing the third column that is not needed for methylation island analysis
!awk '{print $1"\t"$2}' 2019-04-09-All-5x-CpG-Loci-Methylated.bed > 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed

In [ ]:
#Confirm file only has chromosome and start bp for mCpG
!head 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed

### 5b. Identify methylation islands

In [ ]:
#Identify methylation islands using 0.02 mCpG fraction (same as original paper)
! ./methyl_island_sliding_window.pl 500 0.02 50 2019-04-09-All-5x-CpG-Loci-Methylated-Reduced.bed \
> 2020-02-06-Methylation-Islands-500_0.02_50.tab

In [ ]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-500_0.02_50.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-500_0.02_50.tab

In [ ]:
#Filter by MI length and print MI length in a new column
!awk '{if ($3-$2 >= 500) { print $1"\t"$2"\t"$3"\t"$4"\t"$3-$2}}' 2020-02-06-Methylation-Islands-500_0.02_50.tab \
> 2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab
!head 2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab
! wc -l 2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab

In [ ]:
#Count max mCpG in an island
#Count min mCpG in an island
!awk 'NR==1{max = $4 + 0; next} {if ($4 > max) max = $4;} END {print max}' \
2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab
!awk 'NR==1{min = $4 + 0; next} {if ($4 < min) min = $4;} END {print min}' \
2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab

### 5c. Create BEDfiles for IGV and `bedtools`

In [ ]:
#Create tab-delimited BEDfile without additional information
!awk '{print $1"\t"$2"\t"$3}' 2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab \
> 2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab.bed

In [ ]:
#Check output
!head 2020-02-06-Methylation-Islands-200_0.02_50.tab.bed

### 5d. Identify genome feature overlaps with methylation islands

In [ ]:
methylationIslands = "2020-02-06-Methylation-Islands-500_0.02_50-filtered.tab.bed"

In [ ]:
!head {methylationIslands}
!wc -l {methylationIslands}

#### Genes

In [ ]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {geneList} \
| wc -l
!echo "MI overlaps with genes"

In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {geneList} \
> 2020-02-06-MI-Genes.txt

In [ ]:
!head 2020-02-06-MI-Genes.txt

#### Intergenic regions

In [ ]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {methylationIslands} \
-b {intergenic} \
| wc -l
!echo "MI overlaps with intergenic regions"

In [ ]:
! {bedtoolsDirectory}intersectBed \
-wo \
-a {methylationIslands} \
-b {intergenic} \
> 2020-02-06-MI-intergenic.txt

In [ ]:
!head 2020-02-06-MI-intergenic.txt